In [ ]:
# change into the root directory of the project
import os
if os.getcwd().split("/")[-1] == "dev":
    os.chdir('..')

In [ ]:
import logging
logger = logging.getLogger()
#import warnings
#warnings.filterwarnings("ignore")

logger.setLevel(logging.INFO)
#logging.disable(logging.WARNING)
#logging.disable(logging.WARN)

%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['image.cmap'] = 'plasma'
import scipy
from scipy import stats


from neurolib.models.aln import ALNModel
from neurolib.utils.parameterSpace import ParameterSpace
from neurolib.optimize.exploration import BoxSearch
import neurolib.utils.functions as func
import neurolib.utils.devutils as du

import neurolib.utils.brainplot as bp
import neurolib.optimize.exploration.explorationUtils as eu

from neurolib.utils.loadData import Dataset

In [ ]:
plt.style.use("paper")
plt.style.reload_library()

In [ ]:
def peak_isi(peaks):
    return np.diff(peaks)/1000*model.params.dt

In [ ]:
def process_run(output):
    model.outputs[model.default_output] = output
    states = bp.detectSWs(model)
    durations = bp.get_state_lengths(states)
    involvement = bp.get_involvement(states)
    kuramoto_up, kuramoto_down = kuramoto_state_transitions(states, skip=100)
    
    inv = scipy.ndimage.gaussian_filter1d(involvement, 2000)
    peaks25 = scipy.signal.find_peaks(inv, height=0.25, distance=1000)[0]
    peaks50 = scipy.signal.find_peaks(inv, height=0.5, distance=1000)[0]
    peaks75 = scipy.signal.find_peaks(inv, height=0.75, distance=1000)[0]
    peaks = [peaks25, peaks50, peaks75]
    ups, downs = bp.get_state_durations_flat(model, states)
    return states, durations, involvement, kuramoto_up, kuramoto_down, peaks, ups, downs
#process_run(search.results[0]['rates_exc'])

In [1]:
ds = Dataset("gw_big", fcd=False)
model = ALNModel(Cmat = ds.Cmat, Dmat = ds.Dmat)
model.params['dt'] = 0.1
model.params['duration'] = 10.0 * 60 * 1000 #ms

# Results in /mnt/raid/data/cakan/hdf/experiment-10.0-adaptation-noise.hdf
model.params["mue_ext_mean"] = 3.3202829454334535
model.params["mui_ext_mean"] = 3.682451894176651
model.params["b"] = 3.2021806735984186
model.params["tauA"] = 4765.3385276559875
model.params["sigma_ou"] = 0.36802952978628106
model.params["Ke_gl"] = 265.48075753153

NameError: name 'Dataset' is not defined

In [ ]:
import scipy.signal

def filter_peaks(peaks, involvement, threshold_max, threshold_min = 0):
    around = int(500 / model.params.dt)
    #return [p for p in peaks if (np.all(involvement[p-around:p+around] < threshold_max) and np.any(involvement[p-around:p+around]  > threshold_min)) ]
    #return [p for p in peaks if (np.max(involvement[p-around:p+around]) <= threshold_max) \
    #        and (np.max(involvement[p-around:p+around])  > threshold_min) ]
    #return [p for p in peaks if (np.all(involvement[p-around:p+around] <= threshold_max))]
    #print(threshold_min)
    #print([(p, involvement[p]) for p in peaks if involvement[p] > threshold_min])
    return [p for p in peaks if (involvement[p] > threshold_min and involvement[p] <= threshold_max)]

def get_peaks(involvement, min_distance=1000):
    filtered_involvement = scipy.ndimage.gaussian_filter1d(involvement, 2000)
    peaks = scipy.signal.find_peaks(filtered_involvement, height=0.1, distance=min_distance)[0]

    peaks25 = filter_peaks(peaks, involvement, 0.50, 0.0)
    peaks50 = filter_peaks(peaks, involvement, 0.75, 0.50)
    peaks75 = filter_peaks(peaks, involvement, 1, 0.75)

    return peaks25, peaks50, peaks75

def plot_peaks(peaks, color='springgreen'):
    for p in peaks.copy():
        dt_min = p
        around = int(500 / model.params.dt)
        if dt_min+around < len(involvement):
            plt.scatter(dt_min/1000*model.params.dt, involvement[dt_min], color=color, s=5)
            #plt.plot(np.arange(dt_min-around, dt_min+around)/1000*model.params.dt, involvement[dt_min-around : dt_min+around], lw=1, c=color)
    

In [2]:
model.run()
states, durations, involvement, kuramoto_up, kuramoto_down, peaks, ups, downs = process_run(model.output)
inv = scipy.ndimage.gaussian_filter1d(involvement, 2000)
peaks25, peaks50 = get_peaks(inv)

NameError: name 'model' is not defined

In [ ]:
def peak_isi(peaks):
    return np.diff(peaks)/1000*model.params.dt

In [ ]:
# plt.figure(figsize=(6, 3))
# import scipy.signal

# def filter_peaks(peaks, involvement, threshold_max, threshold_min = 0):
#     around = int(500 / model.params.dt)
#     #return [p for p in peaks if (np.all(involvement[p-around:p+around] < threshold_max) and np.any(involvement[p-around:p+around]  > threshold_min)) ]
#     return [p for p in peaks if (np.all(involvement[p-around:p+around] <= threshold_max))]
    
# def get_peaks(involvement):
#     peaks25 = scipy.signal.find_peaks(involvement, height=0.25, distance=1000)[0]
#     peaks50 = scipy.signal.find_peaks(involvement, height=0.50, distance=1000)[0]
#     #peaks75 = scipy.signal.find_peaks(involvement, height=0.75, distance=10000)[0]
    
#     peaks25 = filter_peaks(peaks25, involvement, 0.50, 0.0)
#     peaks50 = filter_peaks(peaks50, involvement, 1.0, 0.50)
#     #peaks75 = filter_peaks(peaks75, involvement, 1, 0.75)
#     return peaks25, peaks50#, peaks75
    
# peaks25, peaks50 = get_peaks(involvement)

# def plot_peaks(peaks, color='springgreen'):
#     for p in peaks:
#         dt_min = p
#         around = int(500 / model.params.dt)
#         if dt_min+around < len(involvement):
#             plt.scatter(dt_min/1000*model.params.dt, involvement[dt_min] + 0.1, color=color, s=3)
#             plt.plot(np.arange(dt_min-around, dt_min+around)/1000*model.params.dt, involvement[dt_min-around : dt_min+around], lw=1, c=color)

# plot_peaks(peaks25, "springgreen")
# plot_peaks(peaks50, "red")
# #plot_peaks(peaks75, "red")

# plt.plot(search.dfResults.t[0]/1000, involvement, c='k', zorder=-2, ls="--", lw=1, alpha=0.2)
# plt.xlabel("t [s]")
# plt.ylabel("Involvement")

In [ ]:
inv = scipy.ndimage.gaussian_filter1d(involvement, 2000)

In [ ]:
peaks25, peaks50 = get_peaks(inv)

In [4]:
plot_peaks(peaks25, "C0")
plot_peaks(peaks50, "C1")
plt.plot(model.t/1000, involvement, c='k', zorder=-2, ls="--", lw=1, alpha=0.2)
plt.xlabel("Time [s]")
plt.ylabel("Involvement")
plt.xlim(0, 100)

involvement = all_involvements[105]    
plt.figure(figsize=(6, 3))

peaks25, peaks50, peaks75 = get_peaks(involvement)

plot_peaks(peaks25, "springgreen")
plot_peaks(peaks50, "dodgerblue")
plot_peaks(peaks75, "red")

plt.hlines([0.25, 0.5, 0.75], 0, 60, linestyles='--', color='darkgray', alpha=0.9)

#plt.savefig("data/figures/RESULTS-adaptation/global_local_waves_timeseries.pdf", transparent=True)
#plt.savefig("data/figures/RESULTS-adaptation/global_local_waves_timeseries.svg", transparent=True)

NameError: name 'plot_peaks' is not defined

In [5]:
figsize=(2, 1.7)
plt.figure(figsize=figsize)
nbins = 25
#plt.hist(all_peak50_isis,alpha=0.75, bins=10**(np.linspace(0,1,nbins)), density=True)
plt.hist(peak_isi(peaks25), rwidth=0.8, edgecolor="k", alpha=1, bins=10**(np.linspace(-1,1,nbins)), density=True, label='Local', color='C0')
plt.hist(peak_isi(np.sort(peaks50 + peaks75).tolist()), rwidth=0.8, edgecolor="k", alpha=0.85, bins=10**(np.linspace(-1,1,nbins)), density=True, label='Global', color='C1')
#plt.legend()
plt.gca().tick_params(axis="both",direction="out",bottom=True,left=True)
plt.xlim(0.35, 10)
plt.xscale('log')
plt.yticks([0, 0.25, 0.5, 0.75], [0, 25, 50, 75])
plt.xticks([1, 10], [1, 10])
plt.text(1.3, 0.55, "local", fontsize=14,  color='C0')
plt.text(2.5, 0.32, "global", fontsize=14,  color='C1')
plt.xlabel("Inter-event interval [s]")
plt.yticks([])
#plt.ylabel("Probability")

ax = plt.gca()
import matplotlib.ticker
locmin = matplotlib.ticker.LogLocator(base=10.0,subs=np.arange(2, 10) * .1, numticks=20)
ax.xaxis.set_minor_locator(locmin)
ax.xaxis.set_minor_formatter(matplotlib.ticker.NullFormatter())
plt.gca().tick_params(axis="x",direction="out",which='minor',bottom=True,left=True)

plt.tight_layout()

ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)
ax.spines["left"].set_visible(False)
#ax.spines["bottom"].set_visible(False)

set_axis_size(*figsize)
#plt.savefig("data/figures/RESULTS-adaptation/global_local_waves_isi.pdf", transparent=True, bbox_inches = "tight")
#plt.savefig("data/figures/RESULTS-adaptation/global_local_waves_isi.svg", transparent=True, bbox_inches = "tight")

NameError: name 'plt' is not defined